# Topic 2. Strongly-typed vs. Dynamic languages

## 2.2 Strongly-typed languages

# Types in Scala

Scala is a statically-typed language, which means that types play a prominent syntactic role and that type-checking is actually enforced by the compiler itself. For instance, we can't declare functions without explicitly indicating the input types:

In [1]:
// Missing signature
def is_even_legnth(s: String) = 
    ??? 

defined function is_even_legnth

> __Scala__: question marks `???` are used to indicate that function body is not finished.

It's mandatory to specify the type of the input arguments: 

In [ ]:
// now with types


The return type is optional (the compiler can infer it - more on this later), but it's good practice to annotate it:

In [ ]:
// now with input and output types
def is_even_length(s: String): Boolean = 
    ??? 

Let's complete the implementation, so that we can move forward:

In [3]:
// full implementation
def is_even_length(s: String): Boolean = 
    //return s.length % 2 == 0
    s.length % 2 == 0

defined function is_even_length

We will explain later on the syntax of the function body, in particular, the  `x.length` expression.

Let's see now the differences with respect to the Python implementation. If we invoke this function with the right arguments, everything is fine:

In [4]:
// right invocations

is_even_length("abcd")
is_even_length("abc")
is_even_length("")

res4_0: Boolean = true
res4_1: Boolean = false
res4_2: Boolean = true

Similarly to the Python implementation, if we invoke the function with a non-string value, an error will be raised:

In [4]:
// wrong input types: type mismatch
is_even_length(0)

-- [E007] Type Mismatch Error: cell5.sc:1:26 -----------------------------------
1 |val res5 = is_even_length(0)
  |                          ^
  |      Found:    (0 : Int)
  |      Required: String
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

This error is a _type error_, as in the Python case: it was expected a value of type `String`, but an `Int` was found. However, this error is raised at _compile time_, not at runtime. This means that the program was not even executed, since "Compilation failed". Note that in the case of Python, the error pointed to the `len` function, meaning that the program was actually being exectued. On the contrary, the Scala compiler raises the error without actually looking at the function body. 

In Scala, there are also type errors that can be reported at runtime, but in order to get those, we have to cheat the compiler. For instance, we can force the compiler to ignore that value `111` is not an integer, but a string, using the `asInstanceOf[String]` expression:

In [6]:
// cheating the compiler
def foo = 
    is_even_length(1.asInstanceOf[String])


defined function foo

In [7]:
foo

java.lang.ClassCastException: java.lang.Integer cannot be cast to java.lang.String

> We write the `is_even_length` expression within the `foo` function in order to prevent the inmediate execution by Jupyter

In doing so, we are actually saying to the compiler: "trust me, this is a string". Then, the compiler will happily trust our judgement and the expression will type-check. However, when we try to execute this program:

In [ ]:
// but when we run this code: ClassCastException



an exception will be raised _at runtime_, reporting the obvious type error. The moral is clear:

> DO NOT USE `asInstanceOf[_]`, it's not type-safe.

During this course, we won't use this unsafe features of the Scala programming language. It's completely forbidden.

This example shows that type ckecks are not only performed by the Scala compiler, but by the generated code as well. On the other hand, note that the error is reported in the form of a `ClassCastException`. Exceptions are the way in which runtime errors are reported, and there are many standard errors. For instance, if we try to divide by 0:

In [8]:
// Another type of runtime error, not catched by at compile-time

def foo: Double = 
    1/0

defined function foo

the compiler won't detect any problem (it could with little effort, by the way), but an `ArithmeticException` will be raised if we attempt to execute that expression:

In [9]:
// But when we run the code 
foo

java.lang.ArithmeticException: / by zero

By the way, note that these exceptions are actually called `java.lang.*`. What does all this have to do with the Java language? Well, a Scala program is actually compiled into Java, and eventually run by the Java Virtual Machine. The standard library of Scala reuses some many features of Java, including the above exceptions, and basic types such as the `java.lang.String` itself.

# Type-inference

Ok, type annotations allow us to anticipate problems at runtime, but it's annoying to have to explicitly annotate our signatures with types. For instance, the returning `Boolean` type of the `is_even_length` function is completely redundant: it's clear from the function body that the function will return a boolean value. 

Fortunately, the Scala compiler is clever enough to perform this kind of type inference itself, and will allow us to get rid of the returned type in this case:

In [11]:
// is_even_length without explicit return type
def is_even_length(s: String) = 
    s.length % 2 == 0

val b: Int = is_even_length("abcd")

-- [E007] Type Mismatch Error: cell12.sc:4:27 ----------------------------------
4 |val b: Int = is_even_length("abcd")
  |             ^^^^^^^^^^^^^^^^^^^^^^
  |      Found:    Boolean
  |      Required: Int
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

However, we will be commonly write the returned type of function signatures, since it will allow us to know what is (partially) doing the function, without actually looking inside the function body. It aids understanding.

# Type-ascription

Besides functions and datatype signatures, expressions can also feature type occurrences. It happens when we call _generic_ functions (e.g. `asInstanceOf[_]`), but also in the case of [_type ascriptions_](https://docs.scala-lang.org/style/types.html#ascription):

In [15]:
// 1+2 with type ascriptions

(1: String) + (2: Int)

-- [E007] Type Mismatch Error: cell16.sc:1:13 ----------------------------------
1 |val res16 = (1: String) + (2: Int)
  |             ^
  |      Found:    (1 : Int)
  |      Required: String
  |
  |      The following import might make progress towards fixing the problem:
  |
  |        import sourcecode.Text.generate
  |
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

With type ascriptions, we are saying to the compiler: "note that value `1` is of type `Int`". In this case, it's completely redundant because the compiler can figure it out itself. In other cases, however, the Scala compiler might struggle without success to find the right type of an expression, and we will have to help it a little bit. This may happen, for instance, when some value have several types because of inheritance.

In [16]:
1 + 3.0

res16: Double = 4.0

# Advantages of static typing

We can summarize the advantages of using types at the source level as follows:

* __Self-explanatory documentation.__ Types do not allow us to dispense with comments, but they certainly reduce the burden. Indeed, type signatures can be regarded as _partial_ specifications of the required functionality, so that comments can focuse in more essential aspects.

* __Prevent errors in advance.__ As Robin Milner put it: "type-checked programs can't go wrong": if the program compiles, it won't raised errors at runtime. Going further, a popular saying in the functional community is: "if it compiles, it _works_".

* __Better aids for IDEs.__ IDEs can leverage type annotations by providing the user with extra functionality such as: showing methods to be used for those types, etc.

* __Type-driven development.__ We will speak about this in long detail in forthcoming lectures.